In [4]:
# verion 5

In [5]:
# imports

from IPython.display import display, Image, clear_output, HTML
import time
import random
import threading
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events
import pandas as pd

import requests
from bs4 import BeautifulSoup
import json

In [6]:
# images, dictionaries, dataframes (defined globally)

image_0 = Image("image_0.png", width = 400)
image_1 = Image("image_1.png", width = 400)
image_2 = Image("image_2.png", width = 400)
image_3 = Image("image_3.png", width = 400)
image_4 = Image("image_4.png", width = 400)
image_5 = Image("image_5.png", width = 400)
    
    
image_list = [image_1, image_2, image_3, image_4, image_5]
    
answer_dict = {image_1 : 'y',
                image_2 : 'y',
                image_3 : 'b',
                image_4 : 'b',
                image_5 : 'y'}


# defining functions

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

# Define mydataframe outside the function scope as a global variable #####################################
collected_data = {
    'username': [],
    'age': [],
    'sex': [],
    'education': [],
    'ANS_history': [],
    'score': [],
    'total_time_taken': [],
}

mydataframe = pd.DataFrame(collected_data)

In [7]:
# functions

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


def demo_ANS_test_2():
    global mydataframe  # Access the global mydataframe variable

    # introduction
    print("Welcome to the Approximate Number System (ANS) test.")
    time.sleep(2)
    print("Before taking the test, please answer the following questions about yourself.")
    time.sleep(2)

    username = input('Enter your username.')
    clear_output()
    age = input('How old are you?')
    clear_output()
    sex = input('What is your biological sex? Enter F for female or M for Male.')
    clear_output()
    education = input('What is the highest level of education you have recieved? (high school diploma, bachelors, masters, doctorate)')
    clear_output()
    ANS_history = input('Have you ever taken an ANS test before?')
    clear_output()
    time.sleep(2)

    print("Thank you for answering the questions. The ANS test will begin soon.")
    time.sleep(2)
    print("When presented with 2 sets (blue set and yellow set),")
    print("choose which set contains more circles by clicking the corresponding button.")
    print("You will have 3 seconds to make a choice.")
    time.sleep(5)
    
    # initialise score and time
    score = 0
    total_time_taken = 0
    total_images = 5
    
    # start of test
    for i in range(total_images):

        image = image_list[i] # defined for use in calling dictionary below (see if statement)
    
        display(image_list[i])
        time.sleep(0.75)
        clear_output(wait=True)
        display(image_0)
        start_time = time.time()
    
        # adding buttons
        btn1 = widgets.Button(description="Blue")
        btn2 = widgets.Button(description="Yellow")
        
        btn1.on_click(register_btn_event)
        btn2.on_click(register_btn_event)
        panel = widgets.HBox([btn1, btn2])
        
        display(panel)
        
        result = wait_for_event(timeout=3)
        
        if result['description'] == 'Blue':
            answer = 'b'
            if answer == answer_dict[image]:
                print("Correct!")
                score = score + 1
            else:
                print("Wrong")
            
        elif result['description'] == 'Yellow':
            answer = 'y'
            if answer == answer_dict[image]:
                print("Correct!")
                score = score + 1
            else:
                print("Wrong")
            
        elif result['description'] == '':
            print("No button was pressed. Answer more quickly.")
            score = score + 0
        
        end_time = time.time()
        time_taken = end_time - start_time
        
        time.sleep(1)
        
        print(f"You took {time_taken:.2f} seconds")
        print(f"Score: {score}")
    
        total_time_taken += time_taken
        total_time_taken = round(total_time_taken)
    
        time.sleep(1.5)
        clear_output(wait=True)

    # storing user information
    newdata = {
        'username': [username],
        'age': [age],
        'sex': [sex],
        'education': [education],
        'ANS_history': [ANS_history],
        'score': [score],
        'total_time_taken': [total_time_taken]
    }

    newdataframe = pd.DataFrame(newdata)
    mydataframe = pd.concat([mydataframe, newdataframe],
                            axis='rows',
                            ignore_index=True)
    
    print("Test over")
    print(f"Your total score is {score} out of {total_images} and total time taken is {total_time_taken} seconds.")

    form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSfNifEg2shqUI7aXrIpEAvmzXXbgeCBezZzoqnY-tXmMwLh-A/viewform?usp=sf_link'
    send_to_google_form(mydataframe, form_url)
    return

In [9]:
demo_ANS_test_2()

Test over
Your total score is 1 out of 5 and total time taken is 13 seconds.
